# El corrido de Gregorio Cortez

## Mapping the manhunt of Rio Grande border folk hero Gregorio Cortez:  the largest manhunt in U.S. history.

### June 14, 1901 to June 22, 1901

### based on lyrics from the Corrido de Gregorio Cortez (variant x english translation by Americo Paredes)

#### project author Diane López 

In [9]:
import geopandas as gpd
import pandas as pd
import numpy as np

from geograpy import places
import re

import geopy
from geopy.geocoders import ArcGIS
#from geopy.extra.rate_limiter import RateLimiter

import shapely
from shapely.geometry import Point
from shapely.wkt import loads

import contextily as cx
import matplotlib.pyplot as plt #to make sure there are no errors when plotting a graph
import pyproj

#import nltk

import spacy
from spacy import displacy

import locationtagger
nlp = spacy.load("en_core_web_sm")

In [ ]:
text = "corrido corpus\ElCorridodeGregorioCortez_X.txt"
with open(text, 'r', encoding='utf-8') as c:
    text = c.read()
    
def clean_text(text):
    cleaned= re.sub(r'[":;,.“”]', "", text)
    return(cleaned)
text = clean_text(text)
#print(text)


TxGPE=[]
nlp = spacy.load("tx_ner_model")
doc =nlp(text) 
#print(doc)
for ent in doc.ents:
    #print(ent.text, ent.label_)
    if ent.label_ == "GPE":
        TxGPE.append(ent.text)
print(TxGPE)

In [ ]:
ents = [(e.text, e.start_char, e.end_char, e.label_)for e in doc.ents]
print(ents)

ents = [(e.text)for e in doc.ents]
print(ents)

In [ ]:
displacy.render(doc, style ='ent', jupyter=True, page=True)

Geoparsing: Finding places from the corrido lyrics

In [ ]:
pc = places.PlaceContext(ents)

pc.set_countries()
print (pc.countries) #['United States']

pc.set_regions()
print(pc.regions) #['Texas'])

pc.set_cities()
print(pc.cities) #['Brownsville'])

#print(pc.address_strings) #['Brownsville','Texas, United States'])
#print(pc.cities[0:5])

Pandas Dataframe
NER Places Geolocation 

In [ ]:
df = pd.DataFrame(ents, columns=["NER_Places"])
geolocator = ArcGIS(user_agent='CorridosMap')
geocode = lambda query: geolocator.geocode("%s, Texas" % query)
df['Location'] = df['NER_Places'].apply(geocode)
df

In [ ]:
gpd = gpd.tools.geocode(df.Location, provider='ArcGIS')
gpd

In [ ]:
ax = gpd.plot(color="red", figsize=(10,10))

#cx.add_basemap(ax, crs=gpd.crs)
cx.add_basemap(ax, crs=gpd.crs, source=cx.providers.OpenStreetMap.Mapnik, zoom=12)
cx.add_basemap(ax, crs=gpd.crs, source=cx.providers.Stamen.TonerLabels, zoom=10)

ax.tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
